# Planning: planning.py; chapters 10-11

This notebook describes the [planning.py](https://github.com/aimacode/aima-python/blob/master/planning.py) module, which covers Chapters 10 (Classical Planning) and  11 (Planning and Acting in the Real World) of *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

We'll start by looking at `PDDL` and `Action` data types for defining problems and actions. Then, we will see how to use them by trying to plan a trip from *Sibiu* to *Bucharest* across the familiar map of Romania, from [search.ipynb](https://github.com/aimacode/aima-python/blob/master/search.ipynb). Finally, we will look at the implementation of the GraphPlan algorithm.

The first step is to load the code:

In [1]:
from planning import *
from notebook import psource

To be able to model a planning problem properly, it is essential to be able to represent an Action. Each action we model requires at least three things:
* preconditions that the action must meet
* the effects of executing the action
* some expression that represents the action

Planning actions have been modelled using the `Action` class. Let's look at the source to see how the internal details of an action are implemented in Python.

In [2]:
%psource Action

It is interesting to see the way preconditions and effects are represented here. Instead of just being a list of expressions each, they consist of two lists - `precond_pos` and `precond_neg`. This is to work around the fact that PDDL doesn't allow for negations. Thus, for each precondition, we maintain a separate list of those preconditions that must hold true, and those whose negations must hold true. Similarly, instead of having a single list of expressions that are the result of executing an action, we have two. The first (`effect_add`) contains all the expressions that will evaluate to true if the action is executed, and the the second (`effect_neg`) contains all those expressions that would be false if the action is executed (ie. their negations would be true).

The constructor parameters, however combine the two precondition lists into a single `precond` parameter, and the effect lists into a single `effect` parameter.

The `PDDL` class is used to represent planning problems in this module. The following attributes are essential to be able to define a problem:
* a goal test
* an initial state
* a set of viable actions that can be executed in the search space of the problem

View the source to see how the Python code tries to realise these.

In [3]:
%psource PDDL

The `initial_state` attribute is a list of `Expr` expressions that forms the initial knowledge base for the problem. Next, `actions` contains a list of `Action` objects that may be executed in the search space of the problem. Lastly, we pass a `goal_test` function as a parameter - this typically takes a knowledge base as a parameter, and returns whether or not the goal has been reached.

Now lets try to define a planning problem using these tools. Since we already know about the map of Romania, lets see if we can plan a trip across a simplified map of Romania.

Here is our simplified map definition:

In [4]:
from utils import *
# this imports the required expr so we can create our knowledge base

knowledge_base = [
    expr("Connected(Bucharest,Pitesti)"),
    expr("Connected(Pitesti,Rimnicu)"),
    expr("Connected(Rimnicu,Sibiu)"),
    expr("Connected(Sibiu,Fagaras)"),
    expr("Connected(Fagaras,Bucharest)"),
    expr("Connected(Pitesti,Craiova)"),
    expr("Connected(Craiova,Rimnicu)")
    ]

Let us add some logic propositions to complete our knowledge about travelling around the map. These are the typical symmetry and transitivity properties of connections on a map. We can now be sure that our `knowledge_base` understands what it truly means for two locations to be connected in the sense usually meant by humans when we use the term.

Let's also add our starting location - *Sibiu* to the map.

In [5]:
knowledge_base.extend([
     expr("Connected(x,y) ==> Connected(y,x)"),
     expr("Connected(x,y) & Connected(y,z) ==> Connected(x,z)"),
     expr("At(Sibiu)")
    ])

We now have a complete knowledge base, which can be seen like this:

In [6]:
knowledge_base

[Connected(Bucharest, Pitesti),
 Connected(Pitesti, Rimnicu),
 Connected(Rimnicu, Sibiu),
 Connected(Sibiu, Fagaras),
 Connected(Fagaras, Bucharest),
 Connected(Pitesti, Craiova),
 Connected(Craiova, Rimnicu),
 (Connected(x, y) ==> Connected(y, x)),
 ((Connected(x, y) & Connected(y, z)) ==> Connected(x, z)),
 At(Sibiu)]

We now define possible actions to our problem. We know that we can drive between any connected places. But, as is evident from [this](https://en.wikipedia.org/wiki/List_of_airports_in_Romania) list of Romanian airports, we can also fly directly between Sibiu, Bucharest, and Craiova.

We can define these flight actions like this:

In [7]:
#Sibiu to Bucharest
precond_pos = [expr('At(Sibiu)')]
precond_neg = []
effect_add = [expr('At(Bucharest)')]
effect_rem = [expr('At(Sibiu)')]
fly_s_b = Action(expr('Fly(Sibiu, Bucharest)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Bucharest to Sibiu
precond_pos = [expr('At(Bucharest)')]
precond_neg = []
effect_add = [expr('At(Sibiu)')]
effect_rem = [expr('At(Bucharest)')]
fly_b_s = Action(expr('Fly(Bucharest, Sibiu)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Sibiu to Craiova
precond_pos = [expr('At(Sibiu)')]
precond_neg = []
effect_add = [expr('At(Craiova)')]
effect_rem = [expr('At(Sibiu)')]
fly_s_c = Action(expr('Fly(Sibiu, Craiova)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Craiova to Sibiu
precond_pos = [expr('At(Craiova)')]
precond_neg = []
effect_add = [expr('At(Sibiu)')]
effect_rem = [expr('At(Craiova)')]
fly_c_s = Action(expr('Fly(Craiova, Sibiu)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Bucharest to Craiova
precond_pos = [expr('At(Bucharest)')]
precond_neg = []
effect_add = [expr('At(Craiova)')]
effect_rem = [expr('At(Bucharest)')]
fly_b_c = Action(expr('Fly(Bucharest, Craiova)'), [precond_pos, precond_neg], [effect_add, effect_rem])

#Craiova to Bucharest
precond_pos = [expr('At(Craiova)')]
precond_neg = []
effect_add = [expr('At(Bucharest)')]
effect_rem = [expr('At(Craiova)')]
fly_c_b = Action(expr('Fly(Craiova, Bucharest)'), [precond_pos, precond_neg], [effect_add, effect_rem])

And the drive actions like this.

In [8]:
#Drive
precond_pos = [expr('At(x)')]
precond_neg = []
effect_add = [expr('At(y)')]
effect_rem = [expr('At(x)')]
drive = Action(expr('Drive(x, y)'), [precond_pos, precond_neg], [effect_add, effect_rem])

Finally, we can define a a function that will tell us when we have reached our destination, Bucharest.

In [9]:
def goal_test(kb):
    return kb.ask(expr("At(Bucharest)"))

Thus, with all the components in place, we can define the planning problem.

In [10]:
prob = PDDL(knowledge_base, [fly_s_b, fly_b_s, fly_s_c, fly_c_s, fly_b_c, fly_c_b, drive], goal_test)

# Air Cargo Problem:

Air Cargo problem involves loading and unloading of cargo and flying it from place to place. The problem can be defined with three actions: Load, Unload and Fly. Let us look at `air_cargo`. 

In [11]:
psource(air_cargo)

**At(x, a):** The cargo or plane **'x'** is at airport **'a'**.

**In(c, p):** Cargo **'c'** is in palne **'p'**.

**Cargo(x):** Declare **'x'** as cargo.

**Plane(x):** Declare **'x'** as plane.

**Airport(x):** Declare **'x'** as airport.



In the `initial_state`, we have cargo C1, plane P1 at airport SFO and cargo C2, plane P2 at airport JFK. Our goal state is to have cargo C1 at airport JFK and cargo C2 at airport SFO. We will discuss on how to achieve this. Let us now define an object of `air_cargo` problem:

In [12]:
airCargo = air_cargo()

Now, before taking any actions, we will check the `airCargo` if it has completed the goal it is required to do:

In [13]:
print(airCargo.goal_test())

False


It returns False because the goal state is not yet reached. Now, we define the sequence of actions that it should take in order to achieve the goal. Then the `airCargo` acts on each of them.

The actions available to us are the following: Load, Unload, Fly

**Load(c, p, a):** Load cargo **'c'** into plane **'p'** from airport **'a'**.

**Fly(p, f, t):** Fly the plane **'p'** from airport **'f'** to airport **'t'**.

**Unload(c, p, c):** Unload cargo **'c'** from plane **'p'** to airport **'a'**.


In [14]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")] 

for action in solution:
    airCargo.act(action)

As the `airCargo` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal:

In [15]:
print(airCargo.goal_test())

True


It has now achieved its goal.

## The Spare Tire Problem

Let's consider the problem of changing a flat tire of a car. The goal is to have a good spare tire properly mounted onto the car's axle, where the initial state has a flat tire on the axle and a good spare tire in the trunk. 

In [16]:
psource(spare_tire)

**At(x, l):** object **'x'** is at location **'l'**.

**Tire(x):** Declare a tire of type **'x'**.

Let us now define an object of `spare_tire` problem:

In [17]:
spare_tire = spare_tire()

Now, before taking any actions, we will check `spare_tire` if it has completed the goal it is required to do

In [18]:
print(spare_tire.goal_test())

False


As we can see, it hasn't completed the goal. Now, we define the sequence of actions that it should take in order to have a good spare tire properly mounted onto the car's axle. Then the `spare_tire` acts on each of them.

The actions available to us are the following: Remove, PutOn

**Remove(obj, loc):** Remove the tire **'obj'** from the location **'loc'**.

**PutOn(t, Axle):** Attach the tire **'t'** on the Axle.



In [19]:
solution = [expr("Remove(Flat, Axle)"),
            expr("Remove(Spare, Trunk)"),
            expr("PutOn(Spare, Axle)")]

for action in solution:
    spare_tire.act(action)

As the `spare_tire` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal

In [20]:
print(spare_tire.goal_test())

True


It has now successfully achieved its goal i.e, to have a good spare tire properly mounted onto the car's axle.

## Three Block Tower Problem

This problem's domain consists of a set of cube-shaped blocks sitting on a table. The blocks can be stacked , but only one block can fit directly on top of another. A robot arm can pick up a block and move it to another position, either on the table or on top of another block. The arm can pick up only one block at a time, so it cannot pick up a block that has another one on it. The goal will always be to build one or more stacks of blocks. In our case, we consider only three blocks.

let us take a look at the `three_block_tower()` code.

In [21]:
psource(three_block_tower)

**On(b, x):** The block **'b'** is on **'x'**. **'x'** can be a table or a block.

**Block(x):** Declares **'x'** as a block.

**Clear(x):** To tell that there is nothing on **'x'**.
 
 Let us now define an object of `three_block_tower` problem:

In [22]:
three_block_tower = three_block_tower()

Now, before taking any actions, we will check `three_tower_block` if it has completed the goal it is required to do

In [23]:
print(three_block_tower.goal_test())

False


As we can see, it hasn't completed the goal. Now, we define the sequence of actions that it should take in order to build a stack of three blocks. Then the `three_block_tower` acts on each of them.

The actions available to us are the following: MoveToTable, Move

**MoveToTable(b, x):** Move the box **'b'** which is on top of box **'x'** to the table.

**Move(b, x, y):** Move box **'b'** from top of **'x'** to the top of **'y'**.


In [24]:
solution = [expr("MoveToTable(C, A)"),
            expr("Move(B, Table, C)"),
            expr("Move(A, Table, B)")]

for action in solution:
    three_block_tower.act(action)

As the `three_block_tower` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal

In [25]:
print(three_block_tower.goal_test())

True


It has now successfully achieved its goal i.e, to build a stack of three blocks.

## Have Cake and Eat Cake Too

This problem involves the task of eating a cake with an initial condition of having a cake. First let us look at take a look at `have_cake_and_eat_cake_too`

In [26]:
psource(have_cake_and_eat_cake_too)

**Have(x):** Declraes that we have **' x '**.

In [27]:
have_cake_and_eat_cake_too = have_cake_and_eat_cake_too()

First let us check wether the goal state (have cake and eat cake) is reached or not.

In [28]:
print(have_cake_and_eat_cake_too.goal_test())

False


As the goal state is not reached we will make some actions and we will let `have_cake_and_eat_cake_too` act on them. To eat the cake we need to bake it. Let us look at the actions that we can do.

**Bake(x):** To bake **' x '**.

**Eat(x):** To eat **' x '**.

In [29]:
solution = [expr("Bake(cake)"),
            expr("Eat(cake)")]

for action in solution:
    have_cake_and_eat_cake_too.act(action)

Now we have made actions to bake the cake and eat the cake. The goal state is **having and eating the cake**. Let us check if it is reached or not.

In [30]:
print(have_cake_and_eat_cake_too.goal_test())

True


It has now successfully achieved its goal i.e, to have and eat the cake.